In [1]:
import pandas as pd
import json
from json import JSONDecodeError

In [2]:
from core.dao import QueryMaker

In [3]:
q = QueryMaker()

In [4]:
q.list_tables()

['gerenciador_dashboard',
 'auditoria_tabela',
 'conteudo',
 'estatistica_acesso',
 'fonte',
 'indicador',
 'indicador_resultado',
 'indicador_tema',
 'nivel_regiao',
 'parametro_sistema',
 'periodo',
 'regiao',
 'tema',
 'tipo_conteudo',
 'tipo_origem',
 'tipo_periodo',
 'tipo_situacao',
 'tipo_unidade_medida',
 'variavel',
 'variavel_indicador',
 'variavel_resultado']

In [5]:
variaveis = q('variavel', limit=None)

In [6]:
variaveis.keys()

Index(['cd_variavel', 'nm_resumido_variavel', 'nm_completo_variavel',
       'dt_atualizacao', 'cd_operador', 'cd_fonte', 'cd_tipo_situacao',
       'nm_apelido_variavel', 'dc_periodo_atualizacao', 'in_variavel_basica',
       'dc_unidade_variavel', 'dc_periodicidade_variavel',
       'dc_unidade_territorial', 'dc_serie_historica', 'dc_nota_tecnica',
       'dc_observacao_variavel', 'tx_fonte_variavel'],
      dtype='object')

In [7]:
q('tipo_situacao')

,cd_tipo_situacao,dc_tipo_situacao,dt_atualizacao,cd_operador
0,1,ATIVO,2018-11-30 15:36:38,21433
1,2,INATIVO,2018-11-30 15:36:38,21433
2,3,EXCLUIDO,2018-11-30 15:36:38,21433


In [8]:
variaveis = variaveis[variaveis['cd_tipo_situacao']==1].copy().reset_index(drop=True)

In [9]:
colunas = ['cd_variavel', 'nm_resumido_variavel', 'nm_completo_variavel',
       'dt_atualizacao', 'tx_fonte_variavel', 'cd_tipo_situacao', 'dc_unidade_territorial']

In [10]:
variaveis = variaveis[colunas].copy()

In [11]:
variaveis['cd_tipo_situacao'].unique()

array([1], dtype=int64)

In [12]:
variaveis.drop('cd_tipo_situacao', axis=1, inplace=True)

In [13]:
variaveis.head()

,cd_variavel,nm_resumido_variavel,nm_completo_variavel,dt_atualizacao,tx_fonte_variavel,dc_unidade_territorial
0,623,V0581,Iniciativas reconhecidas - promoção da inclusã...,2022-01-04 12:35:22.375697,Secretaria Municipal de Direitos Humanos e Cid...,Município
1,192,V0151,Valor liquidado no orçamento da Prefeitura Mun...,2021-12-16 11:20:38.630103,Secretaria Municipal da Fazenda (SMF),Município
2,751,V0709,Número de pessoas em situação de rua,2022-01-11 15:53:11.800363,Secretaria Municipal de Assistência Social (SM...,Município; Distrito; Subprefeitura
3,752,V0710,Adolescentes cumprindo medidas socioeducativas,2022-01-17 13:43:30.992192,Secretaria Municipal de Assistência e Desenvol...,Município
4,753,V0711,Estrangeiros acolhidos na rede socioassistenci...,2022-01-17 13:55:15.038445,Secretaria Municipal de Assistência e Desenvol...,Município


In [14]:
def fonte_as_txt(val):
    
    if pd.isnull(val):
        return ''
    try:
        fonte_arr = json.loads(val)
    except JSONDecodeError:
        val = val.strip()
        if len(val)>1:
            return val
    fonte_arr = json.loads(val)
    noms = [fonte['nm_fonte'] for fonte
            in fonte_arr if fonte.get('nm_fonte')]
    
    return '; '.join(noms)

In [15]:
variaveis['tx_fonte_variavel'] = variaveis['tx_fonte_variavel'].apply(fonte_as_txt)

In [16]:
variaveis['tx_fonte_variavel'].unique()[:10]

array(['Secretaria Municipal de Direitos Humanos e Cidadania (SMDHC)',
       'Secretaria Municipal da Fazenda (SMF)',
       'Secretaria Municipal de Assistência Social (SMADS)',
       'Secretaria Municipal de Assistência e Desenvolvimento Social (SMADS)',
       'Secretaria Municipal de Cultura (SMC); Coordenadoria do Sistema Municipal de Bibliotecas (CSMB)',
       'Secretaria Municipal de Mobilidade e Trânsito (SMT); Companhia de Engenharia de Tráfego (CET); Gestão Urbana SP - Secretaria Municipal de Urbanismo e Licenciamento (SMUL); Câmara Municipal de São Paulo (CMSP)',
       'Fundação Sistema Estadual de Análise de Dados (SEADE)',
       'Secretaria Municipal do Verde e Meio Ambiente (SVMA); CEA - UMAPAZ',
       'Secretaria de Segurança Pública (SSP); Secretaria Municipal de Direitos Humanos e Cidadania (SMDHC)',
       'Pesquisa Origem-Destino Metrô SP; Secretaria Municipal de Mobilidade e Trânsito (SMT)'],
      dtype=object)

In [17]:
indi = q('indicador', limit=None)

In [18]:
indi.keys()

Index(['cd_indicador', 'nm_indicador', 'dc_formula_indicador',
       'dc_conceito_indicador', 'dc_interpretacao_indicador',
       'dc_nota_tecnica', 'in_visibilidade', 'cd_tipo_situacao',
       'dt_atualizacao', 'cd_operador', 'cd_tipo_unidade_medida', 'cd_fonte',
       'nm_completo_indicador', 'dc_periodicidade_indicador',
       'dc_unidade_territorial', 'dc_serie_historica', 'dc_tag_indicador',
       'dc_observacao_indicador', 'tx_fonte_indicador',
       'in_consideracao_regiao'],
      dtype='object')

In [19]:
indi=indi[indi['cd_tipo_situacao']==1].reset_index(drop=True)

In [20]:
nom_vars = set(variaveis['nm_resumido_variavel'])
mapeamento_formulas = {}

for i, row in indi.iterrows():
    formula = row['dc_formula_indicador'] 
    formula = json.loads(formula)
    for item in formula:
        var_name = item.get('caractere')
        if var_name in nom_vars:
            if var_name not in mapeamento_formulas:
                mapeamento_formulas[var_name] = []
            mapeamento_formulas[var_name].append(str(row['cd_indicador']))
        
for var_name, indis in mapeamento_formulas.items():
    
    mapeamento_formulas[var_name] = '; '.join(indis)

In [21]:
variaveis['indicadores'] = variaveis['nm_resumido_variavel'].apply(lambda x:
                                                          mapeamento_formulas.get(x, ''))

In [22]:
results = q('variavel_resultado', limit=None)

In [23]:
results.sample(5)

,cd_sequencia_variavel_resultado,cd_periodo,cd_variavel,cd_regiao,cd_tipo_situacao,cd_tipo_origem,vl_variavel_resultado,dt_atualizacao,cd_do_operador
80178,25686,37,89,112,1,1,731.0000,2019-09-10 12:28:47.070058,None
111756,114994,39,40,4,1,1,8448.0000,2020-06-04 16:45:47.741256,None
55828,3540,27,49,107,1,1,10672.0000,2019-07-10 10:19:40.717373,184147
60384,6392,21,31,173,1,1,11308.0000,2019-06-25 11:45:33.516393,184147
33253,32677,39,122,159,1,1,54.0000,2019-07-12 16:51:30.728414,None


In [24]:
periodos = q('periodo', limit=None)

In [25]:
periodos = periodos[periodos['cd_tipo_situacao']==1].reset_index(drop=True)

In [26]:
de_para_periodos = dict(zip(periodos['cd_periodo'], periodos['vl_periodo']))

In [27]:
mapper_ultimo_periodo = {}

for i, row in variaveis.iterrows():
    
    cd_variavel = row['cd_variavel']
    
    temp_df = results[results['cd_variavel']==cd_variavel].copy()
    temp_df['periodo_num'] = temp_df['cd_periodo'].apply(lambda x: 
                                                         de_para_periodos.get(x, -1))
    temp_df['periodo_num'] = temp_df['periodo_num'].astype(int)
    
    last_period = temp_df['periodo_num'].max()
    
    mapper_ultimo_periodo[cd_variavel] = last_period

In [28]:
variaveis['ultimo_periodo_atualizado'] = variaveis['cd_variavel'].apply(lambda x:
                                                                       mapper_ultimo_periodo.get(x)) 

In [29]:
variaveis.sample(5)

,cd_variavel,nm_resumido_variavel,nm_completo_variavel,dt_atualizacao,tx_fonte_variavel,dc_unidade_territorial,indicadores,ultimo_periodo_atualizado
53,403,V0361,Número de saídas hospitalares por óbito por In...,2021-12-27 15:42:59.164897,Secretaria Municipal da Saúde (SMS),Município,350,2019.0
362,169,V0128,Matrículas - ensino médio todas as redes,2021-11-29 15:59:29.648914,Censo escolar da educação básica do Ministério...,Município,131,2021.0
443,559,V0517,Número de agentes públicos municipais concluin...,2022-09-15 16:47:57.765445,Secretaria de Governo Municipal (SGM),Município,503,2021.0
652,194,V0153,Valor liquidado no orçamento da Prefeitura Mun...,2021-11-29 17:14:46.819674,Secretaria Municipal da Fazenda (SMF),Município,155,2021.0
455,721,V0679,Moradias em setores de risco geológico alto e ...,2022-01-04 17:54:34.166875,Instituto de Pesquisas Tecnológicas do Estado ...,"Subprefeitura, Município",636,2021.0


In [30]:
from config import data_folder
from core.utils.utils import solve_path

In [31]:
fname = solve_path('report_variaveis.xlsx', parent=data_folder)

In [32]:
variaveis.to_excel(fname, index=False)